In [6]:
from agents.structured_chat import StructuredChatAgent
from tools.base import Tool
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# from langchain.utilities import SerpAPIWrapper

# search = SerpAPIWrapper()

# def search_func(args):
#     input = args.get('input')
#     return search.run(input)

# search_tool = Tool(**{
#     "name": "Current Search",
#     "func": search_func,
#     "description": "useful for when you need to answer questions about current events or the current state of the world",
#     "args": {"tool_input": {"type": "string"}}
# })

In [8]:
from pydantic import BaseModel, Field


def company_query_func(args):
    name = (args if isinstance(args, str) else args.get('name')).lower()
    members = {
        "james": "James is a member of esri company",
        "jim": "Jim is a member of google company",
        "jimmy": "Jimmy is a member of openai company",
        "jimothy": "Jimothy is a member facebook company",
    }
    return f"{members[name]}"


class CompanyQuery(BaseModel):
    name: str = Field()


company_query_tool = Tool.from_function(
    name="CompanyQuery",
    func=company_query_func,
    description="useful for when you want to know someone's company",
    args_schema=CompanyQuery
)


def company_info_func(args):
    company = (args if isinstance(args, str) else args.get('company')).lower()
    companys = {
        "esri": "esri is a geo related company",
        "google": "google is a search engine related company",
    }
    return f"{companys[company]}"


class CompanyInfo(BaseModel):
    company: str = Field()


company_info_tool = Tool.from_function(
    name="GetCompanyInfo",
    func=company_info_func,
    description="useful for when you want to know a company's info",
    args_schema=CompanyInfo
)

In [9]:
input = 'James belongs to which company? Introduce the company\'s information'
tools = [company_query_tool, company_info_tool]
agent = StructuredChatAgent(tools, verbose=False, print_prompt=True)

In [10]:
agent.run(input=input)

Respond to the human as helpfully and accurately as possible. You have access to the following tools:

CompanyQuery: useful for when you want to know someone's company, args: {{'name': {{'title': 'Name', 'type': 'string'}}}}
GetCompanyInfo: useful for when you want to know a company's info, args: {{'company': {{'title': 'Company', 'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or CompanyQuery, GetCompanyInfo

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}
```

Begin! Reminder to

'Esri is a geo-related company. They specialize in geographic information system (GIS) software and technology. They provide mapping and spatial analytics solutions for various industries such as government, transportation, utilities, and natural resources. Is there anything specific you would like to know about Esri?'